In [22]:
import tiktoken
import torch

In [23]:
from torch.utils.data import Dataset, DataLoader

In [24]:
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i +1 :i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [25]:
# creating a dataloader:
def dataloaderV1(text, batch_size=4, max_length=256, stride=128, shuffle=False, drop_last=True, num_workers=0):
    tokenizer=tiktoken.get_encoding('gpt2')

    dataset = GPTDatasetV1(text, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=0
    )
    return dataloader

In [26]:
with open('/content/sample_data/verdict_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [27]:
dataloader = dataloaderV1(text=text, batch_size=8, max_length=4, stride=4)

In [28]:
data_iterator = iter(dataloader)

In [29]:
input_tensors, output_tensors = next(data_iterator)
print(f"input_tensors shape: {input_tensors}")
print(f"output_tensors shape: {output_tensors}")

input_tensors shape: tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
output_tensors shape: tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [30]:
# converting token into embeddings:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [31]:
token_embeddings = token_embedding_layer(input_tensors)
token_embeddings.shape

torch.Size([8, 4, 256])

In [32]:
max_length = 4
context_length = max_length

In [33]:
# positional embeddings:
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [34]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [35]:
# final input embeddings:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
